Scripts to tune my models

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import cv2


class SimpleUNet(tf.keras.Model):
    def __init__(self, num_filters=32, depth=4):
        super(SimpleUNet, self).__init__()
        self.num_filters = num_filters
        self.depth = depth

        # Encoder
        self.encoder_blocks = [self.encoder_block(num_filters * 2**i) for i in range(depth)]
        self.flatten_layer = tf.keras.layers.Flatten()  # Add Flatten layer

        # Bottleneck
        self.bottleneck_conv = tf.keras.layers.Conv2D(num_filters * 2**depth, 3, activation='relu', padding='same')

        # Decoder
        self.decoder_blocks = [self.decoder_block(num_filters * 2**(depth - i - 1)) for i in range(depth)]

        # Output convolution
        self.output_conv = tf.keras.layers.Conv2D(1, 1, activation='sigmoid', padding='same')

    def encoder_block(self, filters):
        block = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        ])
        return block

    def decoder_block(self, filters):
        block = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(filters, 2, strides=(2, 2), padding='same'),
            tf.keras.layers.Conv2D(filters, 3, activation='relu', padding='same')
        ])
        return block

    def call(self, inputs):
        x = inputs
        skip_connections = []

        # Encoder
        for block in self.encoder_blocks:
            x = block(x)
            skip_connections.append(x)

        x = self.flatten_layer(x) 
        # Bottleneck
        x = self.bottleneck_conv(x)

        # Decoder
        for i, block in enumerate(self.decoder_blocks):
            x = block(x)
            x = tf.keras.layers.Concatenate()([x, skip_connections[-i - 1]])

        # Output convolution
        x = self.output_conv(x)

        return x


# custom loss functions
def psnr_loss(y_true, y_pred):
    return -tf.image.psnr(y_true, y_pred, max_val=1.0)

param_grid = {
    'num_filters': [32, 64],
    'depth': [3, 4],
    'learning_rate': [0.01, 0.001],
    'optimizer': [Adam, RMSprop]
}
input_folder = '/jupyter/work/fyp/data/img_rec/4th_set_noise/4_noise'
output_folder = '/jupyter/work/fyp/data/img_rec/4th_set_noise/16_noise'


def load_and_preprocess_sinograms(input_folder, output_folder):
    input_images = []
    output_images = []

    input_filenames = sorted(os.listdir(input_folder))
    output_filenames = sorted(os.listdir(output_folder))

    for input_filename, output_filename in zip(input_filenames, output_filenames):
        input_img_path = os.path.join(input_folder, input_filename)
        output_img_path = os.path.join(output_folder, output_filename)

        input_img = cv2.imread(input_img_path, cv2.IMREAD_GRAYSCALE)
        output_img = cv2.imread(output_img_path, cv2.IMREAD_GRAYSCALE)

        if input_img is not None and output_img is not None:
            input_img = cv2.resize(input_img, (128, 128))  
            input_img = input_img.astype('float32') / 255.0  
            input_images.append(input_img)

            output_img = cv2.resize(output_img, (128, 128))  
            output_img = output_img.astype('float32') / 255.0  
            output_images.append(output_img)
        else:
            print(f"Error loading images: {input_img_path}, {output_img_path}")

    if not input_images or not output_images:
        raise ValueError("No valid images found in the input or output folder")

    print("Input images shape:", input_images[0].shape)
    print("Output images shape:", output_images[0].shape)

    # split data into train and test sets
    split_ratio = 0.8  # 80% train, 20% test
    split_index = int(len(input_images) * split_ratio)

    train_input_images, test_input_images = input_images[:split_index], input_images[split_index:]
    train_output_images, test_output_images = output_images[:split_index], output_images[split_index:]

    return train_input_images, train_output_images, test_input_images, test_output_images



def train_evaluate_model(params, train_input_images, train_output_images, test_input_images, test_output_images):
    unet_model = SimpleUNet(num_filters=params['num_filters'], depth=params['depth'])
    optimizer = params['optimizer'](learning_rate=params['learning_rate'])
    unet_model.compile(optimizer=optimizer, loss=psnr_loss, metrics=['accuracy'])

    history = unet_model.fit(train_input_images, train_output_images, epochs=10, batch_size=32, validation_split=0.2)

    loss, accuracy = unet_model.evaluate(test_input_images, test_output_images)

    return loss, accuracy, history.history['loss'], history.history['val_loss']

# grid search
def grid_search(param_grid, train_input_images, train_output_images, test_input_images, test_output_images):
    best_model = None
    best_params = None
    best_loss = float('inf')
    losses = []
    val_losses = []

    for params in ParameterGrid(param_grid):
        print("Training model with parameters:", params)

        loss, _, history_loss, history_val_loss = train_evaluate_model(params, train_input_images, train_output_images, test_input_images, test_output_images)
        losses.append(history_loss)
        val_losses.append(history_val_loss)

        if loss < best_loss:
            best_model = params
            best_loss = loss

    return best_model, losses, val_losses

train_input_images, train_output_images, test_input_images, test_output_images = load_and_preprocess_sinograms(input_folder, output_folder)

# Perform grid search
best_params, losses, val_losses = grid_search(param_grid, train_input_images, train_output_images, test_input_images, test_output_images)

# Print the best parameters
print("Best Parameters:", best_params)

# Plot the losses
plt.figure(figsize=(10, 6))
for i in range(len(losses)):
    plt.plot(losses[i], label=f'Model {i+1} Train')
    plt.plot(val_losses[i], label=f'Model {i+1} Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss for Different Model Configurations')
plt.legend()
plt.grid(True)
plt.show()

Width and depth tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Define U-Net model architecture
def build_unet_model(depth, width, output_size=(128, 128)):
    inputs = tf.keras.Input(shape=(128, 128, 1))
    x = inputs
    # Encoder
    for _ in range(depth):
        x = layers.Conv2D(width, 3, activation='relu', padding='same')(x)
        x = layers.Conv2D(width, 3, activation='relu', padding='same')(x)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        else:
            break  # Skip pooling if input size is too small
    # Bottleneck
    x = layers.Conv2D(width, 3, activation='relu', padding='same')(x)
    # Decoder
    for _ in range(depth):
        x = layers.Conv2DTranspose(width, 2, strides=(2, 2), padding='same')(x)
        x = layers.Conv2D(width, 3, activation='relu', padding='same')(x)
    # Resize output to desired shape
    outputs = layers.Conv2D(1, 1, activation='sigmoid', padding='same')(x)
    outputs = tf.image.resize(outputs, output_size)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=f"sinogram_unet_{width}_{depth}")
    return model



# Define fitness function
def calculate_fitness(model, test_input_images, test_output_images):
    # Evaluate the model
    test_loss = model.evaluate(test_input_images, test_output_images, verbose=0)
    # Calculate Dice coefficient, F1 score, PSNR
    y_pred = model.predict(test_input_images)
    dice = dice_coefficient(test_output_images, y_pred)
    f1 = f1_score(test_output_images, y_pred)
    psnr_val = psnr(test_output_images, y_pred)
    # Combine metrics into a single fitness value (maximize each metric)
    fitness = dice + f1 + psnr_val
    return fitness


# Load and preprocess images (Replace with your actual data loading code)
train_input_images = np.random.rand(100, 128, 128, 1).astype('float32')
train_output_images = np.random.rand(100, 128, 128, 1).astype('float32')
test_input_images = np.random.rand(20, 128, 128, 1).astype('float32')
test_output_images = np.random.rand(20, 128, 128, 1).astype('float32')

# Define parameters for tuning
depth_values = [3, 4, 5, 8]
width_values = [16, 32, 64, 128]

# Placeholder for storing results
results = []

# Loop over depth and width combinations
for depth in depth_values:
    for width in width_values:
        # Build model
        model = build_unet_model(depth, width)
        model.compile(optimizer='adam', loss='mse') s
        history = model.fit(train_input_images, train_output_images, epochs=200, batch_size=32, verbose=0)
        # Calculate fitness
        fitness = calculate_fitness(model, test_input_images, test_output_images)
        results.append((depth, width, fitness))

fig, axes = plt.subplots(len(depth_values), len(width_values), figsize=(15, 10))
for i, depth in enumerate(depth_values):
    for j, width in enumerate(width_values):
        ax = axes[i, j]
        ax.plot(range(10), np.random.rand(10))  # Placeholder data for plotting
        ax.set_title(f"Depth={depth}, Width={width}, Fitness={results[i*len(width_values)+j][2]:.4f}")
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Fitness")

plt.tight_layout()
plt.show()

Structural and algorithmic hyperparameters tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

# U-Net model architecture
def build_unet_model(depth, width, activation, learning_rate, optimizer, output_size=(128, 128)):
    inputs = tf.keras.Input(shape=(128, 128, 1))
    x = inputs
    # Encoder
    for _ in range(depth):
        x = layers.Conv2D(width, 3, activation=activation, padding='same')(x)
        x = layers.Conv2D(width, 3, activation=activation, padding='same')(x)
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        else:
            break  # Skip pooling if input size is too small
    # Bottleneck
    x = layers.Conv2D(width, 3, activation=activation, padding='same')(x)
    # Decoder
    for _ in range(depth):
        x = layers.Conv2DTranspose(width, 2, strides=(2, 2), padding='same')(x)
        x = layers.Conv2D(width, 3, activation=activation, padding='same')(x)
    outputs = layers.Conv2D(1, 1, activation='sigmoid', padding='same')(x)
    outputs = tf.image.resize(outputs, output_size)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=f"sinogram_unet_{width}_{depth}")
    
    # Compile model with provided optimizer and learning rate
    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='mse')
    
    return model

# fitness function
def calculate_fitness(model, test_input_images, test_output_images):
    # Evaluate the model
    test_loss = model.evaluate(test_input_images, test_output_images, verbose=0)
    # Calculate Dice coefficient, F1 score, PSNR
    y_pred = model.predict(test_input_images)
    dice = dice_coefficient(test_output_images, y_pred)
    f1 = f1_score(test_output_images, y_pred)
    psnr_val = psnr(test_output_images, y_pred)
    fitness = dice + f1 + psnr_val
    return fitness

train_input_images = np.random.rand(100, 128, 128, 1).astype('float32')
train_output_images = np.random.rand(100, 128, 128, 1).astype('float32')
test_input_images = np.random.rand(20, 128, 128, 1).astype('float32')
test_output_images = np.random.rand(20, 128, 128, 1).astype('float32')

# Define parameters for tuning
depth_values = [3, 4, 5, 8]
width_values = [16, 32, 64, 128]
activation_functions = ['relu', 'elu', 'selu']
learning_rates = [1e-2, 1e-3, 1e-4]
optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop, tf.keras.optimizers.SGD]

results = []

# Loop over hyperparameter combinations
for depth, width, activation, learning_rate, optimizer in product(depth_values, width_values, activation_functions, learning_rates, optimizers):
    # Build model
    model = build_unet_model(depth, width, activation, learning_rate, optimizer)
    # Train model (Replace with your actual training code)
    history = model.fit(train_input_images, train_output_images, epochs=200, batch_size=32, verbose=0)
    # Calculate fitness
    fitness = calculate_fitness(model, test_input_images, test_output_images)
    results.append((depth, width, activation, learning_rate, optimizer, fitness))

# Print results
for result in results:
    print(f"Depth={result[0]}, Width={result[1]}, Activation={result[2]}, Learning Rate={result[3]}, Optimizer={result[4]}, Fitness={result[5]:.4f}")
    
fig, axes = plt.subplots(len(depth_values), len(width_values), figsize=(15, 10))
for i, depth in enumerate(depth_values):
    for j, width in enumerate(width_values):
        ax = axes[i, j]
        fitness_values = []
        for result in results:
            if result[0] == depth and result[1] == width:
                fitness_values.append(result[5])
        ax.bar(range(len(fitness_values)), fitness_values, color='blue')
        ax.set_title(f"Depth={depth}, Width={width}")
        ax.set_xlabel("Hyperparameter Combination")
        ax.set_ylabel("Fitness")

plt.tight_layout()
plt.show()
